In [160]:
from pycoingecko import CoinGeckoAPI
import datetime as dt
import pandas as pd
import pyodbc

cg = CoinGeckoAPI()

In [161]:
data = cg.get_price(ids='bitcoin', vs_currencies='usd', include_market_cap='true', include_24hr_vol='true', include_24hr_change='true', include_last_updated_at='true')
coinList = cg.get_coins_list(include_platform = 'false')
#dataHist1 = cg.get_coin_history_by_id(id='bitcoin',vs_currencies='usd',date='01-01-2022', localization='false')
dataHist = cg.get_coin_market_chart_range_by_id(id='bitcoin',vs_currency='usd',from_timestamp='1641020400',to_timestamp='1648792799')
#dataHist3 = cg.get_coin_market_chart_by_id(id='bitcoin',vs_currency='usd',days='180')
#print(coinList)
#print(dataHist)
#print(dataHist3)

In [162]:
type(coinList)

list

In [163]:
#Get historical market data include price, market cap, and 24h volume
dcoinList = pd.DataFrame(coinList,
                  columns=['id','symbol', 'name'])
print(dcoinList)

                                   id    symbol                     name
0                              01coin       zoc                   01coin
1            0-5x-long-algorand-token  algohalf       0.5X Long Algorand
2       0-5x-long-altcoin-index-token   althalf  0.5X Long Altcoin Index
3      0-5x-long-ascendex-token-token   asdhalf       0.5X Long AscendEx
4        0-5x-long-bitcoin-cash-token   bchhalf   0.5X Long Bitcoin Cash
...                               ...       ...                      ...
13390                        zynecoin       zyn                 Zynecoin
13391                            zyro      zyro                     Zyro
13392                           zyrri       zyr                    Zyrri
13393                   zytara-dollar      zusd            Zytara Dollar
13394                             zyx       zyx                      ZYX

[13395 rows x 3 columns]


In [164]:
type(dataHist)

dict

In [165]:
	#Get historical market data include price, market cap, and 24h volume
ddataHist = dataHist ["prices"]
#print(dataHist)

In [166]:
ddataHist = pd.DataFrame(ddataHist, columns =['timestampp', 'priceUSD'])
ddataHist.sort_values(by='timestampp') 
print(ddataHist)

         timestampp      priceUSD
0     1641020554453  47128.848463
1     1641024444418  47292.778947
2     1641028659948  47251.883398
3     1641031594501  47223.103990
4     1641035011717  46983.080704
...             ...           ...
2155  1648774924255  45557.572208
2156  1648778560739  45086.747368
2157  1648782128779  44347.204444
2158  1648785652514  44543.548925
2159  1648789316372  44719.711080

[2160 rows x 2 columns]


In [167]:
ddataHist.insert(2, "idcoin", "btc", allow_duplicates=False)
print(ddataHist)

         timestampp      priceUSD idcoin
0     1641020554453  47128.848463    btc
1     1641024444418  47292.778947    btc
2     1641028659948  47251.883398    btc
3     1641031594501  47223.103990    btc
4     1641035011717  46983.080704    btc
...             ...           ...    ...
2155  1648774924255  45557.572208    btc
2156  1648778560739  45086.747368    btc
2157  1648782128779  44347.204444    btc
2158  1648785652514  44543.548925    btc
2159  1648789316372  44719.711080    btc

[2160 rows x 3 columns]


In [168]:
print(ddataHist.dtypes)

timestampp      int64
priceUSD      float64
idcoin         object
dtype: object


In [169]:
# Convert entire DataFrame to string
ddataHist=ddataHist.applymap(str)
print(ddataHist.dtypes)

timestampp    object
priceUSD      object
idcoin        object
dtype: object


In [ ]:
##################################################################

In [170]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=FREYJA\SQLEXPRESS;'
                      'Database=tradingCrypto;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM [dbo].[cryptoPrices_raw]')

for i in cursor:
    print(i)

In [171]:
# Insert Dataframe into SQL Server:
for index, row in ddataHist.iterrows():
     cursor.execute("INSERT INTO dbo.cryptoPrices_raw (timestampp,id_coin,priceUSD) values(?,?,?)", row.timestampp, row.idcoin, row.priceUSD)
conn.commit()
cursor.close()
# see total time to do insert
#print("%s seconds ---" % (time.time() - start_time))